## 📊 Documentation: Benchmarking a Scenario on One Layout

The library provides functionality to **benchmark wildfire scenarios** using customizable **drone routing** and **sensor placement** strategies. This allows users to evaluate the performance of different approaches under a specific layout and scenario.

---

### 🛠️ Function: `run_benchmark_scenario(...)`

Benchmarks a single wildfire scenario on a given layout using user-specified strategies and simulation parameters.

### **Function Signature**

```python
run_benchmark_scenario(
    scenario,
    sensor_placement_strategy,
    drone_routing_strategy,
    custom_initialization_parameters_function,
    custom_step_parameters_function,
    simulation_parameters
)


In [14]:
# Import necessary modules
import os
import numpy as np
import sys

module_path = os.path.abspath("..") + "/code"
if module_path not in sys.path:
    sys.path.append(module_path)
    print(f"Added {module_path} to sys.path")

from dataset import load_scenario_npy
from benchmark import run_benchmark_scenario
from wrappers import wrap_log_sensor_strategy, wrap_log_drone_strategy
from Strategy import RandomSensorPlacementStrategy, DroneRoutingUniformMaxCoverageResetStatic
from new_clustering import get_wrapped_clustering_strategy

# Define paths and parameters
scenario_file = "../MinimalDataset/0001/burn_map.npy"
scenario = load_scenario_npy(scenario_file)

# Define simulation parameters
simulation_parameters = {
    "max_battery_distance": -1,
    "max_battery_time": 1,
    "n_drones": 3,
    "n_ground_stations": 4,
    "n_charging_stations": 2,
    "drone_speed_m_per_min": 600,
    "coverage_radius_m": 300,
    "cell_size_m": 30,
    "transmission_range": 50000,
}

# Define strategies
sensor_strategy = wrap_log_sensor_strategy(RandomSensorPlacementStrategy)
drone_strategy = wrap_log_drone_strategy(get_wrapped_clustering_strategy(DroneRoutingUniformMaxCoverageResetStatic))

# Define custom initialization parameters
def custom_initialization_parameters_function(input_dir: str):
    return {
        "burnmap_filename": f"{input_dir.strip('/')}/burn_map.npy",  # Correct path construction
        "reevaluation_step": 5,
        "optimization_horizon": 10,
        "regularization_param": 1,
    }

# Run benchmark on the scenario
results, _ = run_benchmark_scenario(
    scenario,
    sensor_strategy,
    drone_strategy,
    custom_initialization_parameters_function("../MinimalDataset/0001/"), 
    lambda: {},  # No custom step parameters
    simulation_parameters=simulation_parameters,
    return_history=True
)

# Print benchmark results
print("Benchmark Results:")
print(results)

Creating initial routing model (reusable)
--- parameters for julia (Julia indexing) ---
burnmap_filename: ../MinimalDataset/0001/burn_map_rescaled_5x14_substeps_63.npy
n_drones: 3
charging_stations_locations: [(1, 11), (3, 12)]
ground_sensor_locations: [(1, 1), (1, 13), (4, 2), (5, 8)]
optimization_horizon: 10


NameError: name 'jl' is not defined

### Expected Output
A dictionary containing benchmark results, such as:

avg_execution_time: Average time taken for the simulation.

fire_size_cells: Total size of the fire in cells.

percentage_map_explored: Percentage of the map explored by drones.

total_distance_traveled: Total distance traveled by drones.


In [16]:

os.environ["JULIA_EXE"] = "/Applications/Julia-1.10.app/Contents/Resources/julia/bin/julia"  # Replace with actual path

from juliacall import Main as jl

print(jl.eval("1 + 1"))  # Should print 2


[juliapkg] Found dependencies: /Users/josephye/Library/Python/3.9/lib/python/site-packages/juliapkg/juliapkg.json
[juliapkg] Found dependencies: /Users/josephye/Library/Python/3.9/lib/python/site-packages/juliacall/juliapkg.json
[juliapkg] Locating Julia 1.6.1 - 1.10.0, ^1.10.3
[juliapkg] Using Julia 1.11.5 at /Users/josephye/.julia/environments/pyjuliapkg/pyjuliapkg/install/bin/julia
[juliapkg] Using Julia project at /Users/josephye/.julia/environments/pyjuliapkg
[juliapkg] Writing Project.toml:
             [deps]
             PythonCall = "6099a3de-0909-46bc-b1f4-468b9a2dfc0d"
             OpenSSL_jll = "458c3c95-2e84-50aa-8efc-19380b2a3a95"
             [compat]
             PythonCall = "=0.9.25"
             OpenSSL_jll = "=2.0.0"
[juliapkg] Installing packages:
             import Pkg
             Pkg.Registry.update()
             Pkg.add([
               Pkg.PackageSpec(name="PythonCall", uuid="6099a3de-0909-46bc-b1f4-468b9a2dfc0d"),
               Pkg.PackageSpec(name="OpenSS

KeyboardInterrupt: 